In [2]:
#1.1 Initial datapoints. log loss 0.64
#1.2 Momentum factors, CPI/PPI Calculations, modified timeframe. LL: 1yr - 10.64 3M - 1.64
#1.3 Standardizing continuous features
#1.4 Inserting Linear Splines

In [3]:
#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import Pipeline

from basis_expansions.basis_expansions import (
    Polynomial, LinearSpline)

from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

from sklearn.preprocessing import StandardScaler

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [4]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [5]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [6]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [7]:
#converting GDP quarterly data into monthly

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [8]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)

In [9]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [10]:
#we split off the recession data here because later we start purging 0s
y = dataset['Recession'] #splitting off Y
dataset = dataset.drop(columns = ['GDP','Recession'])

In [11]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    dataclean.clean_zeros(col, dataset)

In [12]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [13]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [14]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
dataset = dataset.iloc[552:]
dataset = dataset.iloc[:-59]

In [15]:
dataset.head()

,PMI,UNR,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,...,PERS_SAVINGS_PXY_12m_shift,EXPORTS_PXY_12m_shift,IMPORTS_PXY_12m_shift,TRADE_BALANCE_PXY_12m_shift,CPI_1m_shift,PPI_1m_shift,CPI_3m_shift,PPI_3m_shift,CPI_12m_shift,PPI_12m_shift
1959-01,64.4,6.0,11.6,90.8,1901.731975,1022.0,93.706113,754.261755,1657.0,39.375999,...,-1.0,0.0,0.0,0.0,0.346021,0.316456,0.346021,0.634921,1.398601,0.634921
1959-02,66.9,5.9,11.1,90.8,1901.731975,973.0,93.706113,754.261755,1667.0,39.375999,...,-1.0,0.0,0.0,0.0,-0.344828,0.000000,-0.344828,0.316456,1.048951,0.634921
1959-03,67.1,5.6,11.1,90.8,1901.731975,1102.0,93.706113,754.261755,1620.0,39.375999,...,-1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.316456,0.347222,0.000000
1959-04,66.9,5.2,10.8,90.8,1901.731975,1086.0,93.706113,754.261755,1590.0,39.375999,...,-1.0,0.0,0.0,0.0,0.346021,0.315457,0.000000,0.315457,0.346021,0.632911
1959-05,68.2,5.1,10.0,95.3,1901.731975,968.0,93.706113,754.261755,1498.0,39.375999,...,-1.0,0.0,0.0,0.0,0.000000,0.000000,0.346021,0.315457,0.346021,0.315457


In [16]:


#spline time - splines seriously impede the model, Time Horizon of 1 goes from LL of 4.9 to 8.19, AUC degreades by .04
#stickiness remains

#individual splines

CPPR_PRICE_fit = Pipeline([
    ('CPPR_PRICE', ColumnSelector(name='CPPR_PRICE')),
    ('CPPR_PRICE_spline', LinearSpline(knots=[160]))
])

Spread_fit = Pipeline([
    ('Spread', ColumnSelector(name='Spread')),
    ('Spread_spline', LinearSpline(knots=[0,0.25]))
])

EXPORTS_1m_shift_fit = Pipeline([
    ('EXPORTS_1m_shift', ColumnSelector(name='EXPORTS_1m_shift')),
    ('EXPORT1m_spline', LinearSpline(knots=[700,900]))
])

ThreeYT_1m_shift_fit = Pipeline([
    ('3YT_1m_shift', ColumnSelector(name='3YRT_1m_shift')),
    ('3YT_1m_spline', LinearSpline(knots=[-15]))
])

US_M2_1m_shift = Pipeline([
    ('US_M2_1m_shift', ColumnSelector(name='US_M2_1m_shift')),
    ('US_M2_1m_spline', LinearSpline(knots=[17]))
])

HOME_SALES_3m_shift = Pipeline([
    ('HOME_SALES_3m_shift', ColumnSelector(name='HOME_SALES_3m_shift')),
    ('HOME_SALES_3m_spline', LinearSpline(knots=[-500]))
])

PART_TIME_3m_shift = Pipeline([
    ('PART_TIME_3m_shift', ColumnSelector(name='PART_TIME_3m_shift')),
    ('PART_TIME_3m_spline', LinearSpline(knots=[-160]))
])

CAP_UTIL_3m_shift = Pipeline([
    ('CAP_UTIL_3m_shift', ColumnSelector(name='CAP_UTIL_3m_shift')),
    ('CAP_UTIL_3m_spline', LinearSpline(knots=[-0.8]))
])

EXPORTS_3m_shift = Pipeline([
    ('EXPORTS_3m_shift', ColumnSelector(name='EXPORTS_3m_shift')),
    ('EXPORTS_3m_spline', LinearSpline(knots=[1500,1600]))
])

IMPORTS_3m_shift = Pipeline([
    ('IMPORTS_3m_shift', ColumnSelector(name='IMPORTS_3m_shift')),
    ('IMPORTS_3m_spline', LinearSpline(knots=[2000]))
])

TRADE_BALANCE_3m_shift = Pipeline([
    ('TRADE_BALANCE_3m_shift', ColumnSelector(name='TRADE_BALANCE_3m_shift')),
    ('TRADE_BALANCE_3m_spline', LinearSpline(knots=[-2500]))
])

US_M2_3m_shift = Pipeline([
    ('US_M2_3m_shift', ColumnSelector(name='US_M2_3m_shift')),
    ('US_M2_3m_spline', LinearSpline(knots=[60]))
])

HOME_SALES_12m_shift = Pipeline([
    ('HOME_SALES_12m_shift', ColumnSelector(name='HOME_SALES_12m_shift')),
    ('HOME_SALES_12m_spline', LinearSpline(knots=[60]))
])

PART_TIME_12m_shift = Pipeline([
    ('PART_TIME_12m_shift', ColumnSelector(name='PART_TIME_12m_shift')),
    ('PART_TIME_12m_spline', LinearSpline(knots=[-225, -187.5, -180,-140]))
])

CPPR_PRICE_12m_shift = Pipeline([
    ('CPPR_PRICE_12m_shift', ColumnSelector(name='CPPR_PRICE_12m_shift')),
    ('CPPR_PRICE_12m_spline', LinearSpline(knots=[-30]))
])

CAP_UTIL_12m_shift = Pipeline([
    ('CAP_UTIL_12m_shift', ColumnSelector(name='CAP_UTIL_12m_shift')),
    ('CAP_UTIL_12m_spline', LinearSpline(knots=[-2]))
])

Spread_12m_shift = Pipeline([
    ('Spread_12m_shift', ColumnSelector(name='Spread_12m_shift')),
    ('Spread_12m_spline', LinearSpline(knots=[-1]))
])





#union features together

feature_pipeline = FeatureUnion([
    ('intercept', Intercept()),
    ('CPPR_PRICE_fit', CPPR_PRICE_fit),
    ('Spread_fit', Spread_fit),
    ('EXPORTS_1m_shift_fit', EXPORTS_1m_shift_fit),
    ('ThreeYT_1m_shift_fit', ThreeYT_1m_shift_fit),
    ("US_M2_1m_shift", US_M2_1m_shift),
    ("HOME_SALES_3m_shift", HOME_SALES_3m_shift),
    ("PART_TIME_3m_shift", PART_TIME_3m_shift),
    ("CAP_UTIL_3m_shift", CAP_UTIL_3m_shift),
    ("EXPORTS_3m_shift", EXPORTS_3m_shift),
    ("IMPORTS_3m_shift", IMPORTS_3m_shift),
    ("TRADE_BALANCE_3m_shift", TRADE_BALANCE_3m_shift),
    ("HOME_SALES_12m_shift", HOME_SALES_12m_shift),
    ("PART_TIME_12m_shift", PART_TIME_12m_shift),
    ("CPPR_Price_12m_shift", CPPR_PRICE_12m_shift),
    ("CAP_UTIL_12m_shift", CAP_UTIL_12m_shift),
    ("Spread_12m_shift", Spread_12m_shift)
])


feature_pipeline.fit(dataset)
features = feature_pipeline.transform(dataset)

In [17]:
#dropping columns from OG dataset that were splined

splined_cols = ['CPPR_PRICE','Spread','EXPORTS_1m_shift','3YRT_1m_shift','US_M2_1m_shift','PART_TIME_3m_shift',
'CAP_UTIL_3m_shift',
'EXPORTS_3m_shift',
'IMPORTS_3m_shift',
'TRADE_BALANCE_3m_shift',
'US_M2_3m_shift',
'HOME_SALES_12m_shift',
'PART_TIME_12m_shift',
'CPPR_PRICE_12m_shift',
'CAP_UTIL_12m_shift',
'Spread_12m_shift']

dataset = dataset.drop(columns = splined_cols)

In [18]:
#merge splined features into dataset

#dataset = dataset.join(features, how='outer')

In [19]:
list(dataset.columns)

['PMI',
 'UNR',
 'YUNR',
 'CONS_SENT',
 'HOME_SALES',
 'PART_TIME',
 'HOUS_PERMS',
 'HOUS_STARTS',
 'CAP_UTIL',
 'PERS_SAVINGS',
 'EXPORTS',
 'IMPORTS',
 'TRADE_BALANCE',
 'INT_RATE',
 'US_M2',
 'US_NHOME_SALES',
 'PPI',
 'CPI',
 '3YRT',
 '10 Year Treasury Yield',
 '3 Month Treasury Yield (Bond Equivalent Basis)',
 'PMI_PXY',
 'UNR_PXY',
 'YUNR_PXY',
 'CONS_SENT_PXY',
 'HOME_SALES_PXY',
 'PART_TIME_PXY',
 'CPPR_PRICE_PXY',
 'HOUS_PERMS_PXY',
 'HOUS_STARTS_PXY',
 'CAP_UTIL_PXY',
 'PERS_SAVINGS_PXY',
 'EXPORTS_PXY',
 'IMPORTS_PXY',
 'TRADE_BALANCE_PXY',
 'INT_RATE_PXY',
 'US_M2_PXY',
 'US_NHOME_SALES_PXY',
 'PPI_PXY',
 'CPI_PXY',
 '3YRT_PXY',
 'PMI_1m_shift',
 'UNR_1m_shift',
 'YUNR_1m_shift',
 'CONS_SENT_1m_shift',
 'HOME_SALES_1m_shift',
 'PART_TIME_1m_shift',
 'CPPR_PRICE_1m_shift',
 'HOUS_PERMS_1m_shift',
 'HOUS_STARTS_1m_shift',
 'CAP_UTIL_1m_shift',
 'PERS_SAVINGS_1m_shift',
 'IMPORTS_1m_shift',
 'TRADE_BALANCE_1m_shift',
 'INT_RATE_1m_shift',
 'US_NHOME_SALES_1m_shift',
 '10 Year 

In [74]:
#not standardizing as splined columns are now in there

"""

#standardizing in the logistic regression model specifically as that's the only model it will impact

stand_cols = ['PMI',
 'UNR',
 'YUNR',
 'CONS_SENT',
 'HOUS_PERMS',
 'HOUS_STARTS',
 'CAP_UTIL',
 'PERS_SAVINGS',
 'INT_RATE',
 'PPI',
 'CPI',
 '3YRT',
 '10 Year Treasury Yield',
 '3 Month Treasury Yield (Bond Equivalent Basis)',
 'Spread',
 'PMI_1m_shift',
 'UNR_1m_shift',
 'YUNR_1m_shift',
 'CONS_SENT_1m_shift',
 'HOUS_PERMS_1m_shift',
 'HOUS_STARTS_1m_shift',
 'CAP_UTIL_1m_shift',
 'PERS_SAVINGS_1m_shift',
 'INT_RATE_1m_shift',
 '3YRT_1m_shift',
 '10 Year Treasury Yield_1m_shift',
 '3 Month Treasury Yield (Bond Equivalent Basis)_1m_shift',
 'Spread_1m_shift',
 'PMI_3m_shift',
 'UNR_3m_shift',
 'YUNR_3m_shift',
 'CONS_SENT_3m_shift',
 'HOUS_PERMS_3m_shift',
 'HOUS_STARTS_3m_shift',
 'CAP_UTIL_3m_shift',
 'PERS_SAVINGS_3m_shift',
 'INT_RATE_3m_shift',
 '3YRT_3m_shift',
 '10 Year Treasury Yield_3m_shift',
 '3 Month Treasury Yield (Bond Equivalent Basis)_3m_shift',
 'Spread_3m_shift',
 'PMI_12m_shift',
 'UNR_12m_shift',
 'YUNR_12m_shift',
 'CONS_SENT_12m_shift',
 'HOUS_PERMS_12m_shift',
 'HOUS_STARTS_12m_shift',
 'CAP_UTIL_12m_shift',
 'PERS_SAVINGS_12m_shift',
 'INT_RATE_12m_shift',
 '3YRT_12m_shift',
 '10 Year Treasury Yield_12m_shift',
 '3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift',
 'Spread_12m_shift']

dataset[stand_cols] = StandardScaler().fit_transform(dataset[stand_cols])
#dataset[list(dataset.columns)] = StandardScaler().fit_transform(list(dataset.columns))

"""


"\n\n#standardizing in the logistic regression model specifically as that's the only model it will impact\n\nstand_cols = ['PMI',\n 'UNR',\n 'YUNR',\n 'CONS_SENT',\n 'HOUS_PERMS',\n 'HOUS_STARTS',\n 'CAP_UTIL',\n 'PERS_SAVINGS',\n 'INT_RATE',\n 'PPI',\n 'CPI',\n '3YRT',\n '10 Year Treasury Yield',\n '3 Month Treasury Yield (Bond Equivalent Basis)',\n 'Spread',\n 'PMI_1m_shift',\n 'UNR_1m_shift',\n 'YUNR_1m_shift',\n 'CONS_SENT_1m_shift',\n 'HOUS_PERMS_1m_shift',\n 'HOUS_STARTS_1m_shift',\n 'CAP_UTIL_1m_shift',\n 'PERS_SAVINGS_1m_shift',\n 'INT_RATE_1m_shift',\n '3YRT_1m_shift',\n '10 Year Treasury Yield_1m_shift',\n '3 Month Treasury Yield (Bond Equivalent Basis)_1m_shift',\n 'Spread_1m_shift',\n 'PMI_3m_shift',\n 'UNR_3m_shift',\n 'YUNR_3m_shift',\n 'CONS_SENT_3m_shift',\n 'HOUS_PERMS_3m_shift',\n 'HOUS_STARTS_3m_shift',\n 'CAP_UTIL_3m_shift',\n 'PERS_SAVINGS_3m_shift',\n 'INT_RATE_3m_shift',\n '3YRT_3m_shift',\n '10 Year Treasury Yield_3m_shift',\n '3 Month Treasury Yield (Bond Equiv

In [21]:
X = dataset

In [291]:
### Data Prep Finished Here ###

In [22]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
y = y.iloc[552:]
y = y.iloc[:-59]

In [60]:
y_shift = y.shift(-12) #shifting y to forecast 3 months out

In [61]:
y_shift = y_shift.fillna(0)

In [62]:
X_train = X.iloc[12:550]
X_test = X.iloc[550:]
y_train = y_shift.iloc[12:550]
y_test = y_shift.iloc[550:]

In [63]:
X_train.columns

Index(['PMI', 'UNR', 'YUNR', 'CONS_SENT', 'HOME_SALES', 'PART_TIME',
       'HOUS_PERMS', 'HOUS_STARTS', 'CAP_UTIL', 'PERS_SAVINGS',
       ...
       'PERS_SAVINGS_PXY_12m_shift', 'EXPORTS_PXY_12m_shift',
       'IMPORTS_PXY_12m_shift', 'TRADE_BALANCE_PXY_12m_shift', 'CPI_1m_shift',
       'PPI_1m_shift', 'CPI_3m_shift', 'PPI_3m_shift', 'CPI_12m_shift',
       'PPI_12m_shift'],
      dtype='object', length=138)

In [64]:
X_train.shape

(538, 138)

In [65]:
model = LogisticRegression() #try throwing in a bigger C than 1
model.fit(X_train, y_train) #fitting model

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [66]:
probs = model.predict_proba(X_test)

In [67]:
log_loss(y_test, probs)

7.680868276948863

In [68]:
roc_auc_score(y_test.values, probs[:,1:])

0.6504424778761062

In [69]:
for object in zip(dataset.columns,model.coef_[0] ):
    print(object)

('PMI', -0.06212580855998783)
('UNR', -0.02182598524751315)
('YUNR', -0.0467437006218272)
('CONS_SENT', 0.08466379133940298)
('HOME_SALES', 0.0017018555025905123)
('PART_TIME', -0.028031704085346092)
('HOUS_PERMS', -0.01734143927141689)
('HOUS_STARTS', 0.016615388094269413)
('CAP_UTIL', -0.11189237764778137)
('PERS_SAVINGS', -0.03926777948865298)
('EXPORTS', -0.0021321121159573924)
('IMPORTS', 0.0008260081068674768)
('TRADE_BALANCE', 0.0015449938758774619)
('INT_RATE', 0.1155421721294783)
('US_M2', 0.029510087224953428)
('US_NHOME_SALES', -0.05582504018378961)
('PPI', 0.14903675083965182)
('CPI', 0.017494873881471612)
('3YRT', 0.147250368488352)
('10 Year Treasury Yield', 0.0017128023229038387)
('3 Month Treasury Yield (Bond Equivalent Basis)', 0.09584647740764297)
('PMI_PXY', 0.0)
('UNR_PXY', 0.0)
('YUNR_PXY', 0.0)
('CONS_SENT_PXY', 0.0)
('HOME_SALES_PXY', -0.001202346909812514)
('PART_TIME_PXY', 0.0)
('CPPR_PRICE_PXY', -0.00015877242280905383)
('HOUS_PERMS_PXY', 0.0)
('HOUS_STARTS_PX

In [70]:
results = pd.DataFrame(probs)
results['actual'] = y_test.values
results.index = y_test.index
pd.set_option('display.float_format', lambda x: '%.3f' % x)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(results)

            0     1  actual
2004-11 1.000 0.000   0.000
2004-12 1.000 0.000   0.000
2005-01 1.000 0.000   0.000
2005-02 1.000 0.000   0.000
2005-03 1.000 0.000   0.000
2005-04 1.000 0.000   0.000
2005-05 1.000 0.000   0.000
2005-06 1.000 0.000   0.000
2005-07 1.000 0.000   0.000
2005-08 1.000 0.000   0.000
2005-09 1.000 0.000   0.000
2005-10 1.000 0.000   0.000
2005-11 1.000 0.000   0.000
2005-12 1.000 0.000   0.000
2006-01 1.000 0.000   0.000
2006-02 1.000 0.000   0.000
2006-03 1.000 0.000   0.000
2006-04 1.000 0.000   0.000
2006-05 1.000 0.000   0.000
2006-06 0.986 0.014   0.000
2006-07 1.000 0.000   0.000
2006-08 1.000 0.000   0.000
2006-09 1.000 0.000   0.000
2006-10 1.000 0.000   0.000
2006-11 0.940 0.060   0.000
2006-12 1.000 0.000   0.000
2007-01 0.000 1.000   1.000
2007-02 1.000 0.000   1.000
2007-03 0.076 0.924   1.000
2007-04 0.993 0.007   0.000
2007-05 0.100 0.900   0.000
2007-06 0.040 0.960   0.000
2007-07 0.004 0.996   0.000
2007-08 0.000 1.000   0.000
2007-09 0.000 1.000 

In [71]:
#Testing on the training data to see if model fit got better
probs = model.predict_proba(X_train)

In [72]:
log_loss(y_train, probs)

0.012927259322249461

In [73]:
roc_auc_score(y_train.values, probs[:,1:])

1.0